# Création des Embeddings et Vector Stores

Ce notebook permet de créer et persister les embeddings de vos documents pour le système RAG.

## Objectif

- Charger vos documents (DOCX)
- Les découper en chunks avec un séparateur personnalisé pour les codes SH
- Créer les embeddings avec HuggingFace
- Persister les vector stores dans ChromaDB (séparés par type : import/export)

## 1. Configuration de l'environnement

In [ ]:
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis un fichier .env
# Créez un fichier .env avec vos clés API (voir ENV_SETUP.txt)
load_dotenv()

# Configuration LangSmith (optionnel - pour le tracing)
# Définissez LANGCHAIN_API_KEY dans votre fichier .env si vous souhaitez utiliser LangSmith
if os.getenv("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
## 2. Imports des bibliothèques


## 3. Configuration et traitement des documents

Cette section configure le modèle d'embedding, le text splitter, charge les documents, les découpe en chunks et crée les vector stores.

In [ ]:
# Imports nécessaires pour le traitement des documents et la création des embeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

import os

# ============================================================================
# Configuration
# ============================================================================

# Modèle d'embedding HuggingFace (BAAI/bge-large-en-v1.5 pour meilleure qualité)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

# Séparateur personnalisé pour les codes SH (Système Harmonisé)
# Ce pattern détecte les codes SH dans les documents (ex: 1234.56, 123456)
CODE_SH_SEPARATOR = r"\b(?:\d{6,8}|\d{4}(?:\.\d{2,4})?)\b"

# Configuration du text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,          # Taille maximale d'un chunk
    chunk_overlap=100,        # Chevauchement entre chunks pour préserver le contexte
    add_start_index=True,     # Ajouter l'index de début dans les métadonnées
    strip_whitespace=True,    # Supprimer les espaces en début/fin
    separators=[CODE_SH_SEPARATOR],  # Utiliser le séparateur personnalisé
)

# ============================================================================
# Chargement des documents
# ============================================================================
# Remplacez ces chemins par les chemins vers vos propres documents
# Format recommandé : placez vos documents dans un dossier "documents/" à la racine

IMPORT_DOC_PATH = "documents/import_document.docx"  # Remplacez par votre chemin
EXPORT_DOC_PATH = "documents/export_document.docx"  # Remplacez par votre chemin

# Charger les documents
loader_import = Docx2txtLoader(IMPORT_DOC_PATH)
loader_export = Docx2txtLoader(EXPORT_DOC_PATH)

docs_import = loader_import.load()
docs_export = loader_export.load()

print(f"Documents import chargés : {len(docs_import)} pages")
print(f"Documents export chargés : {len(docs_export)} pages")

# ============================================================================
# Découpage en chunks
# ============================================================================

chunks_import = splitter.split_documents(docs_import)
chunks_export = splitter.split_documents(docs_export)

print(f"Chunks import créés : {len(chunks_import)}")
print(f"Chunks export créés : {len(chunks_export)}")

# ============================================================================
# Ajout de métadonnées
# ============================================================================
# Les métadonnées permettent de filtrer et router les requêtes vers le bon vector store

for chunk in chunks_import:
    chunk.metadata["type"] = "import"
    
for chunk in chunks_export:
    chunk.metadata["type"] = "export"

# ============================================================================
# Création et persistance des vector stores
# ============================================================================

# Créer les dossiers de stockage si nécessaire
os.makedirs("chroma_store/import", exist_ok=True)
os.makedirs("chroma_store/export", exist_ok=True)

# Créer et persister le vector store pour les documents d'import
vs_import = Chroma.from_documents(
    chunks_import,
    embedding_model,
    persist_directory="chroma_store/import",
    collection_name="import_docs"
)
vs_import.persist()

# Créer et persister le vector store pour les documents d'export
vs_export = Chroma.from_documents(
    chunks_export,
    embedding_model,
    persist_directory="chroma_store/export",
    collection_name="export_docs"
)
vs_export.persist()

print("\n✅ Embeddings et vector stores créés et persistés avec succès!")
print("   - Vector store import : chroma_store/import")
print("   - Vector store export : chroma_store/export")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]